In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import copy

from collections import defaultdict 

from synthesizer import Synthesizer
from generator import Generator
from algorithm import Algorithm

# warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 80

### Эксперимент

In [2]:
# seeds = 5 * np.arange(2, 6) + 111

#### Фиксированные параметры генератора
- series_type = "default"  // обычная линейная регрессия

- dim=10 // размерность сигнала
- low=-10, high=10 // границы весов для получения отклика
- noise_var=1 // дисперсия нормального шума
- workers_num=3 // число различных генераторов
- lower_bound=100, upper_bound=400 // границы длин каждого из кусков, получаемых генератором
- alternating=True // нет двух подряд кусков одного типа
#### Фиксированные параметры алгоритма
- from_start=False // начинаем подсчет всех потерь (и составного эксперта) не с самого начала, а после того как прошли по одному разу учатки каждого из  генераторов

- length=1500 // длина изучаемого участка временного ряда

- a=-40, b=40 // параметры алгоритма, отклики отсеиваем тоже по ним

#### Гиперпараметры эксперимента
- windows = [5, 10, 20] // варьируем длину окна обучения ($\frac12x, 1x, 2x$ соответственно, где $x$  &mdash; dim  сигнала )
- weights_function // функция начальных весов, cравниваем $\dfrac1{x^\alpha}$ для разных $\alpha$ и дефолтную $\dfrac{1}{(x+1)\ln^2(x+1)}$
- alpha_function // функция коэффицента $\alpha$ в Mixing Update, cравниваем $\dfrac1{(1 + x)^\alpha}$ для разных $\alpha$

In [3]:
from hypers import weight_hypers
from hypers import alpha_hypers 

from main import experiments
from main import df

100%|█████████████████████████████████████████████████████████████████████████| 223/223 [00:00<00:00, 410.13it/s]


In [4]:
df

,windows,weight_function,alpha_function,mean,random_1,random_2,random_3,random_4
1,10,1 / (x^1.01),1 / (x + 1),28413.14,24665.09,23763.2,26267.51,38956.77
4,10,1 / (x^1.01),1 / (x + 1)^0.1,32871.58,31333.98,30000.99,28393.74,41757.63
0,10,1 / (x^1.01),1 / (x + 1)^0.5,28597.43,25366.92,25933.93,25085.51,38003.35
2,10,1 / (x^1.01),1 / (x + 5),28431.08,24689.37,23777.47,26284.94,38972.54
3,10,1 / (x^1.01),1 / (x + 25),28517.11,24807.39,23847.02,26364.02,39050.0


In [5]:
df_w = df[df["alpha_function"] == "1 / (x + 1)"].iloc[:, :4]
df_w

,windows,weight_function,alpha_function,mean
1,10,1 / (x^1.01),1 / (x + 1),28413.14


In [6]:
df_a = df[df["weight_function"] == "1 / (x^1.01)"].iloc[:,:4]
df_a

,windows,weight_function,alpha_function,mean
1,10,1 / (x^1.01),1 / (x + 1),28413.14
4,10,1 / (x^1.01),1 / (x + 1)^0.1,32871.58
0,10,1 / (x^1.01),1 / (x + 1)^0.5,28597.43
2,10,1 / (x^1.01),1 / (x + 5),28431.08
3,10,1 / (x^1.01),1 / (x + 25),28517.11


In [7]:
df[["train_window", "mean"]].groupby("train_window").mean()

KeyError: "['train_window'] not in index"